In [1]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import requests

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [2]:
#we need to use a user agent here, because metacritics does not accept bots
#we are using a tested go-to user agent here, it can be found online
#read more about user agents here:
#https://www.scrapehero.com/how-to-fake-and-rotate-user-agents-using-python-3/
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

In [3]:
url = input('Enter - ')
page = requests.get(url, headers = headers)
soup = BeautifulSoup(page.content, "html.parser")
# https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc

Enter - https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc


In [4]:
container = soup.find_all('td', class_ = 'clamp-summary-wrap')
print(type(container))

<class 'bs4.element.ResultSet'>


In [5]:
#then lets find out how many movies we got
len(container)

100

In [6]:
movie_names = []
movies = container[:50] #here we get the top 50 movies we want
movieData = []
for movie in movies:
    mnameList = []
    name = movie.find('h3').text
    movie_names.append(name)
    movieData.append(mnameList)
    mnameList.append(name)
print(movie_names)

['Citizen Kane', 'The Godfather', 'Rear Window', 'Casablanca', 'Boyhood', 'Three Colors: Red', 'Vertigo', 'Notorious', "Singin' in the Rain", 'City Lights', 'Moonlight', 'Intolerance', 'Pinocchio', 'Touch of Evil', 'The Treasure of the Sierra Madre', "Pan's Labyrinth", 'Some Like It Hot', 'North by Northwest', 'Hoop Dreams', 'Rashomon', 'All About Eve', 'Jules and Jim', 'The Wild Bunch', 'My Left Foot', 'The Third Man', 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb', 'Nomadland', 'Gone with the Wind', '4 Months, 3 Weeks and 2 Days', 'Psycho', 'Battleship Potemkin', 'A Streetcar Named Desire', 'American Graffiti', 'Dumbo', 'Roma', 'Ran', 'The Shop Around the Corner', '12 Angry Men', 'Manchester by the Sea', "Rosemary's Baby", 'The Maltese Falcon', '12 Years a Slave', 'Killer of Sheep', 'Rocks', 'Nashville', 'Ratatouille', 'Parasite', "Don't Look Now", 'The Grapes of Wrath', 'Children of Paradise (1945)']


In [7]:
len(movie_names)

50

In [37]:
#now we need to click into each movie name in order to see detailed information
#so we need to extract all the urls
movie_names = []
movies = container[:50] #here we get the top 50 movies we want
movieData = []
parentUrl = "https://www.metacritic.com"
review_links = []
    
for movie in movies:
    movieDict = {}
    name = movie.find('h3').text
    movie_names.append(name)
    
    movieDict["Movie Name"] = name
#     movieData.append(movieDict)

    tag = movie.find('a', class_ = 'title')
    link = tag.get('href', None)
    
    #there are a few different ways to get the links to user reviews
    #lets first try to write a loop to open all 50 links of these movies
    
    detailsLink = parentUrl+link
    page = requests.get(detailsLink, headers = headers)
    soup = BeautifulSoup(page.content, "html.parser")
    tag = soup.find('a', class_ = 'see_all boxed oswald')
    review_link = tag.get('href', None)
    review_links.append(review_link)
    
    review_page = requests.get(detailsLink + '/user-reviews', headers = headers)
    soup = BeautifulSoup(review_page.content, "html.parser")
    tag = soup.find_all('div', class_ = 'summary')
#     print(len(tag))
    top_50 = tag[:50]
#     print(len(top_50))
    reviewsList = []
    for i in range(len(top_50)):
        s = top_50[i].text
        s = re.sub(r'[^\w\s]','',s)
        reviewsList.append(s.strip())
# print(reviewsList[0])
    movieDict["Reviews"] = reviewsList
    review_page = requests.get(detailsLink, headers = headers)
    soup = BeautifulSoup(review_page.content, "html.parser")
    genres = soup.find_all('div', class_ = 'genres')
    
    genres = genres[1:]
    
    genList = []
    for genre in genres:
        gen = genre.find('span')
        print(gen.text)
    movieDict["Genre"] = gen.text
    
    movieData.append(movieDict)
    
# print(review_links)
# https://www.metacritic.com/movie/citizen-kane-1941
# https://www.metacritic.com/movie/citizen-kane-1941/user-reviews
print(movieData)

NameError: name 'gen' is not defined

In [30]:
n = 0
for i in range(len(review_links)):
    if not review_links[i].endswith('user-reviews'):
        continue
    else:
        review_page = requests.get('https://www.metacritic.com'+review_links[i], headers = headers)
        soup = BeautifulSoup(review_page.content, "html.parser")
        tag = soup.find_all('div', class_ = 'summary') #find_all to return a list
        n = n + 1
#         print('Movie', n)
        print('Total user reviews:', len(tag))
        top_50 = tag[:50]
        print(len(top_50))#some of the movies don't have 50 reviews, but that is fine  
        print(top_50)

In [34]:
import re
        
for i in range(len(review_links)):
    review_page = requests.get('https://www.metacritic.com'+review_links[i], headers = headers)
    soup = BeautifulSoup(review_page.content, "html.parser")
    tag = soup.find_all('div', class_ = 'summary')
#     print(len(tag))
    top_50 = tag[:50]
#     print(len(top_50))
    reviewsList = []
    for i in range(len(top_50)):
        s = top_50[i].text
        s = re.sub(r'[^\w\s]','',s)
        reviewsList.append(s.strip())
# print(reviewsList[0])
    movieData[i]["Reviews"] = reviewsList
print(movieData[0:2])

[{'Movie Name': 'Citizen Kane'}, {'Movie Name': 'The Godfather'}]


In [38]:
import csv
finalData = []
print(len(movieData))
for i in range(len(movieData)):
    
    movieDict = {}
    name = movieData[i][0]
#     genre = movieData[i][1]
    revs = movieData[i][1]
    movieDict["Movie Name"] = name
#     movieDict["Genre"] = genre
    movieDict["Reviews"] = revs
    finalData.append(movieDict)
# print(movieDict)
    
csv_file = "metacritic.csv"
csv_columns = ["Movie Name", "Genre", "Reviews"]
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        count = 1
#         print(finalData)
        for data in finalData:
            writer.writerow(data)
#             print(count)
            count+=1
    print("Data saved to metacritic.csv")

except IOError:
    print("I/O error")

0
Data saved to metacritic.csv


In [ ]:
# storing data in SQLite
import pandas as pd
import sqlite3

conn = sqlite3.connect('meta.db')
c = conn.cursor()
c.execute('DROP TABLE IF EXISTS movies')
c.execute('''CREATE TABLE movies (movie_name text, genre text, review text)''')


# load the data into a Pandas DataFrame
movies = pd.read_csv('meta.csv')
# movies = movies.drop(label)

# write the data to a sqlite table
# movies.head(2)

# for row in movies.iterrows:
for index, row in movies.iterrows(): 
#     print (row["Movie Name"]) 
    name = row["Movie Name"]
    genre = row["Genre"]
    review = row["Reviews"]
    
    c.execute('INSERT INTO movies (movie_name, genre, review) VALUES (?,?,?)',
                       (name, genre, review))
    conn.commit()
    print(name, genre)
c.close()
conn.close()
print("Data saved to meta.db")
